# Samuel Moraes - TP3
## TP3
Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en 
Material_TPs/TP3/images a partir del template Material_TPs/TP3/template

In [148]:
%matplotlib qt

import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from IPython.display import Image, display


### 1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos


In [149]:
# Cargamos la imagen a procesar y el template
img = cv.imread('images/coca_logo_1.png')

img_rgb= cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_gray= cv.cvtColor(img, cv.COLOR_BGR2GRAY)

template = cv.imread('template/pattern.png',0)
#template_inv = cv.cvtColor('template/pattern.png', cv.THRESH_BINARY_INV)

def imshow(img, ax=None):
    if ax is None:
        ret, encoded = cv.imencode(".jpg", img)
        display(Image(encoded))
    else:
        ax.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        ax.axis('off')
 
# Procesamiento de color invertido para detección 
ret, template = cv.threshold(template,
                             0, 255, 
                             cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

# Redimensionar la imagen
def rescale_image(image):
    scale_percent = 300  # Porcentaje del tamaño original
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)

    return dim

resized_template = cv.resize(template, rescale_image(template), interpolation=cv.INTER_LINEAR)

template_height, template_width = template.shape


plt.figure()
plt.subplot(1,2,1)
plt.imshow(resized_template, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_rgb, cmap='gray')

In [150]:
def detect_logo(logo_image, image_path, template_height, template_width, scales=[0.2, 0.3, 0.4, 0.7, 0.8, 0.9, 1.0, 1.2, 1.5, 1.7, 1.8, 1.9, 2.0, 2.1, 2.5, 3.0, 3.2, 3.5, 4.0], threshold=0.5):
    # Carrega as imagens
    image = cv.imread(image_path)
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    image_height, image_width = image.shape

    # Chequea si las imagens fueran cargadas con exito
    if logo_image is None or image is None:
        print("Error al cargar imagenes.")
        return False

    found = None
    count = 0
    old_resized_logo = None

    # Probando las escalas para un mejor match
    # El template necesita tener las dimensiones menores que las imagenes para hacer la detección, entonces vamos aplicar un downscale:
    for scale in scales:

        if count == 0:
            old_scale = scale
            print(f"resized scale: {scale}")
            resized_logo = cv.resize(logo_image, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
        
        old_resized_logo = resized_logo
        old_template_height, old_template_width = resized_logo.shape
        
        if count > 0:
            print(f"resized scale: {scale}")
            resized_logo = cv.resize(logo_image, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)

        template_height, template_width = resized_logo.shape
        count += 1
        if image_height <= template_height or image_width <= template_width:
            print("stop")
            if old_resized_logo is not None:
                template_height, template_width = old_resized_logo.shape
                result = cv.matchTemplate(image, old_resized_logo, cv.TM_CCORR_NORMED)
                min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
                if found is None or max_val > found[0]:
                    resized_logo = cv.resize(logo_image, None, fx=old_scale, fy=old_scale, interpolation=cv.INTER_LINEAR)
                    print(f"resized old scale: {old_scale}")
                    found = (max_val, max_loc, scale, resized_logo.shape[1], resized_logo.shape[0])
                count = 0
            else:
                result = cv.matchTemplate(image, resized_logo, cv.TM_CCORR_NORMED)
                min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
            
                if found is None or max_val > found[0]:
                    print("entrou")
                    found = (max_val, max_loc, scale, resized_logo.shape[1], resized_logo.shape[0])
                count = 0
            break

        old_scale = scale

    max_val, max_loc, scale, w, h = found

    if max_val >= threshold:
        print("Logotipo encontrado na imagem.")
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        result_image = cv.cvtColor(image, cv.COLOR_GRAY2BGR)
        cv.rectangle(result_image, top_left, bottom_right, (0, 255, 0), 2)

        # Plota a imagem resultante
        result_image_rgb = cv.cvtColor(result_image, cv.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 8))
        plt.imshow(result_image_rgb)
        plt.title("Logotipo Detectado")
        plt.axis('off')
        plt.show()
        return True
    else:
        print("Logotipo não encontrado na imagem.")
        return False

# Exemplo de uso
logo_image_path = "template/pattern.png"
image_path = "images/coca_logo_1.png"

logo_presente = detect_logo(template, image_path, template_height, template_width)


resized scale: 0.2
resized scale: 0.3
resized scale: 0.4
resized scale: 0.7
stop
resized old scale: 0.4
Logotipo encontrado na imagem.


In [151]:
def Identify_logos_pipeline(directory_path):
    try:
        # Validate if the provided path exists
        if not os.path.exists(directory_path):
            print(f"Error: Directorio '{directory_path}' no existe.")
            return

        # Iterate through all files in the directory
        file_count = 0
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isfile(file_path):
                file_count += 1
                # Process each image file (you can add your custom logic here)
                print(f"Procesando imagen: {filename}")

                detect_logo(template, file_path, template_height, template_width)

        print("Procesamiento completo para todos los archivos.")
    except Exception as e:
        print(f"Error: {str(e)}")

# Example usage
images_path = 'images/' 
Identify_logos_pipeline(images_path)

Procesando imagen: COCA-COLA-LOGO.jpg
resized scale: 0.2
resized scale: 0.3
resized scale: 0.4
resized scale: 0.7
resized scale: 0.8
resized scale: 0.9
resized scale: 1.0
resized scale: 1.2
resized scale: 1.5
resized scale: 1.7
resized scale: 1.8
resized scale: 1.9
resized scale: 2.0
resized scale: 2.1
resized scale: 2.5
resized scale: 3.0
resized scale: 3.2
resized scale: 3.5
stop
resized old scale: 3.2
Logotipo encontrado na imagem.
Procesando imagen: coca_logo_1.png
resized scale: 0.2
resized scale: 0.3
resized scale: 0.4
resized scale: 0.7
stop
resized old scale: 0.4
Logotipo encontrado na imagem.
Procesando imagen: coca_logo_2.png
resized scale: 0.2
resized scale: 0.3
resized scale: 0.4
resized scale: 0.7
stop
resized old scale: 0.4
Logotipo encontrado na imagem.
Procesando imagen: coca_multi.png
resized scale: 0.2
resized scale: 0.3
resized scale: 0.4
resized scale: 0.7
resized scale: 0.8
resized scale: 0.9
resized scale: 1.0
resized scale: 1.2
resized scale: 1.5
resized scale: 1

In [ ]:
def Identify_logos_pipeline(directory_path):
    try:
        # Validar si camino existe
        if not os.path.exists(directory_path):
            print(f"Error: Directorio '{directory_path}' no existe.")
            return

        # Iterar por todos los archivos de imagen
        file_count = 0
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isfile(file_path):
                file_count += 1
                # Processar cada archivo
                print(f"Procesando imagen: {filename}")

                detect_logo(template, file_path, template_height, template_width)

        print("Procesamiento completo para todos los archivos.")
    except Exception as e:
        print(f"Error: {str(e)}")

# Example usage
images_path = 'images/' 
Identify_logos_pipeline(images_path)

Procesando imagen: COCA-COLA-LOGO.jpg
resized scale: 0.2
resized scale: 0.3
resized scale: 0.4
resized scale: 0.7
resized scale: 0.8
resized scale: 0.9
resized scale: 1.0
resized scale: 1.2
resized scale: 1.5
resized scale: 1.7
resized scale: 1.9
resized scale: 2.0
resized scale: 2.1
resized scale: 2.5
resized scale: 3.0
resized scale: 3.5
stop
resized old scale: 3.0
Logotipo encontrado na imagem.
Procesando imagen: coca_logo_1.png
resized scale: 0.2
resized scale: 0.3
resized scale: 0.4
resized scale: 0.7
stop
resized old scale: 0.4
Logotipo encontrado na imagem.
Procesando imagen: coca_logo_2.png
resized scale: 0.2
resized scale: 0.3
resized scale: 0.4
resized scale: 0.7
stop
resized old scale: 0.4
Logotipo encontrado na imagem.
Procesando imagen: coca_multi.png
resized scale: 0.2
resized scale: 0.3
resized scale: 0.4
resized scale: 0.7
resized scale: 0.8
resized scale: 0.9
resized scale: 1.0
resized scale: 1.2
resized scale: 1.5
resized scale: 1.7
resized scale: 1.9
resized scale: 2

### 2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen coca_multi.png con el mismo témplate del ítem 1

In [146]:
def detect_multi_logo(logo_image, image_path, template_height, template_width, threshold, scales=[0.1, 0.2, 0.23, 0.24]):
    # Cargar las imagenes
    image = cv.imread(image_path)
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    image_height, image_width = image.shape

    if logo_image is None or image is None:
        print("Erro ao carregar as imagens.")
        return False

    found = None
    count = 0
    total_count = len(scales)
    total_count -= 1
    old_resized_logo = None
    for scale in scales:

        if count == 0:
            old_scale = scale
            print(f"resized scale: {scale}")
            resized_logo = cv.resize(logo_image, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
        
        old_resized_logo = resized_logo
        old_template_height, old_template_width = resized_logo.shape
        
        if count > 0:
            print(f"resized scale: {scale}")
            resized_logo = cv.resize(logo_image, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)

        template_height, template_width = resized_logo.shape
        count += 1
        if (image_height <= template_height or image_width <= template_width) or count == total_count:
            print("stop")
            if old_resized_logo is not None:
                template_height, template_width = old_resized_logo.shape
                result = cv.matchTemplate(image, old_resized_logo, cv.TM_CCORR_NORMED)
                min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
                if found is None or max_val > found[0]:
                    resized_logo = cv.resize(logo_image, None, fx=old_scale, fy=old_scale, interpolation=cv.INTER_LINEAR)
                    print(f"resized old scale: {old_scale}")
                    found = (max_val, max_loc, scale, resized_logo.shape[1], resized_logo.shape[0])
                count = 0
            else:
                result = cv.matchTemplate(image, resized_logo, cv.TM_CCORR_NORMED)
                min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
            
                if found is None or max_val > found[0]:
                    print("entrou")
                    found = (max_val, max_loc, scale, resized_logo.shape[1], resized_logo.shape[0])
                count = 0
            break

        old_scale = scale

    max_val, max_loc, scale, w, h = found

    if max_val >= threshold:
        print("Logotipo encontrado na imagem.")
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        result_image = cv.cvtColor(image, cv.COLOR_GRAY2BGR)

        loc = np.where( result >= threshold)
        for pt in zip(*loc[::-1]):
            cv.rectangle(result_image, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

        # Plot de imagen resultante
        result_image_rgb = cv.cvtColor(result_image, cv.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 8))
        plt.imshow(result_image_rgb)
        plt.title("Logo Detectado")
        plt.axis('off')
        plt.show()
        return True
    else:
        print("Logotipo no encontrado en la imagen.")
        return False

detect_multi_logo(template, "images\coca_multi.png", template_height, template_width, 0.55)

resized scale: 0.1
resized scale: 0.2
resized scale: 0.23
stop
resized old scale: 0.2
Logotipo encontrado na imagem.


True


3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección